In [3]:
import pandas as pd
import numpy as np
import sys, os

import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from scipy.stats import norm

In [4]:
options = pd.read_csv("data/options.csv")

options['mid_price'] = (options.ask_price + options.bid_price) * 0.5
options['otype'] = options.option_type.map({
    "C" : 1,
    "P" : -1
})

options = options[options.implied_volatility != 0]
options = options[options.bid_price != 0]
options = options[options.ask_price != 0]
options = options[abs(options.stock_price / options.strike_price - 1) <= 0.35]

options = options[options.tdays_to_expiry > 10]
options = options[options.tdays_to_expiry < 400]
options['tdays_to_expiry'] = options.tdays_to_expiry / 252

options['dexpd'] = options.date_current + " " + options.expiration_date

start = "2010-01-01"
end = "2030-01-01"
fridays = pd.date_range(start, end, freq="WOM-3FRI").astype(str)
thursdays = pd.date_range(start, end, freq="WOM-3THU").astype(str)

regulars = list(fridays) + list(thursdays)
options = options[options.expiration_date.isin(regulars)]

options.head()

,date_current,ticker,expiration_date,days_to_expiry,option_id,option_type,strike_price,bid_price,option_price,ask_price,...,open_interest,stock_price,dividend_yield,tdays_to_expiry,rate,bivs,nivs,mid_price,otype,dexpd
1254,2019-11-06,SPY,2019-12-20,44,SPY 2019-12-20 C228,C,228.0,79.73,72.52,80.03,...,30,307.07,1.81,0.123016,1.758964,0.516525,0.516525,79.880,1,2019-11-06 2019-12-20
1255,2019-11-06,SPY,2019-12-20,44,SPY 2019-12-20 C229,C,229.0,78.73,72.51,79.03,...,10,307.07,1.81,0.123016,1.758964,0.510006,0.510006,78.880,1,2019-11-06 2019-12-20
1256,2019-11-06,SPY,2019-12-20,44,SPY 2019-12-20 C230,C,230.0,77.73,78.05,78.04,...,3,307.07,1.81,0.123016,1.758964,0.504025,0.504025,77.885,1,2019-11-06 2019-12-20
1257,2019-11-06,SPY,2019-12-20,44,SPY 2019-12-20 C231,C,231.0,76.73,76.92,77.04,...,4,307.07,1.81,0.123016,1.758964,0.497548,0.497548,76.885,1,2019-11-06 2019-12-20
1258,2019-11-06,SPY,2019-12-20,44,SPY 2019-12-20 C232,C,232.0,75.74,63.12,76.05,...,2,307.07,1.81,0.123016,1.758964,0.492102,0.492102,75.895,1,2019-11-06 2019-12-20


In [5]:
x = options.dexpd.value_counts()
x = x[x >= 100]
options = options[options.dexpd.isin(x.index)]

In [6]:
def implied_forward(exp):
    
    cols = ['strike_price', 'mid_price']
    calls = exp[exp.option_type == "C"][cols]
    puts = exp[exp.option_type == "P"][cols]
    
    fprice = calls.merge(puts, on='strike_price', how='inner')
    fprice = fprice.reset_index(drop=True)
    fprice = fprice.mid_price_x - fprice.mid_price_y
    
    S = exp.stock_price.values[0]
    r = exp.rate.values[0]
    T = exp.tdays_to_expiry.values[0]
    
    return S + np.exp(-r * T) * fprice[abs(fprice).argmin()]
    
fprices = options.groupby("dexpd").apply(implied_forward)
fprices = fprices.reset_index(name="forward_price")
options = options.merge(fprices, on="dexpd", how="inner")
options = options[options.otype * (options.forward_price - options.strike_price) < 0]

In [7]:
def atm_iv(exp):
    
    cols = ['strike_price', 'bivs']
    
    calls = exp[exp.option_type == "C"][cols]
    calls = calls.sort_values("strike_price", ascending=True)
    
    puts = exp[exp.option_type == "P"][cols]
    puts = puts.sort_values("strike_price", ascending=True)

    return 0.5 * (calls.bivs.values[0] + puts.bivs.values[-1])

atm_ivs = options.groupby("dexpd").apply(atm_iv)
atm_ivs = atm_ivs.reset_index(name="atm_iv")
options = options.merge(atm_ivs, on="dexpd", how="inner")

In [8]:
atm_ivs.head(5)

,dexpd,atm_iv
0,2019-11-06 2019-12-20,0.119577
1,2019-11-06 2020-01-17,0.124232
2,2019-11-06 2020-02-21,0.135608
3,2019-11-06 2020-03-20,0.145434
4,2019-11-06 2020-06-19,0.154657


In [9]:
def adjust_ivs(exp):
        
    cols = ['strike_price', 'bivs']
    calls = exp[exp.option_type == "C"][cols]
    calls = calls.sort_values("strike_price", ascending=True)
    civ = calls.bivs.values[0]
    
    puts = exp[exp.option_type == "P"][cols]
    puts = puts.sort_values("strike_price", ascending=True)
    piv = puts.bivs.values[-1]
    
    atm_iv = exp.atm_iv.values[0]
    if piv >= civ:        
        adj_factor = piv - atm_iv
        exp['adj_bivs'] = exp.bivs + adj_factor * exp.otype
    elif piv < civ:
        adj_factor = civ - atm_iv
        exp['adj_bivs'] = exp.bivs - adj_factor * exp.otype
        
    return exp

options = options.groupby("dexpd").apply(adjust_ivs)

In [10]:
r = np.log(options.rate + 1).values
q = np.log(options.dividend_yield / 100 + 1).values
F = options.forward_price.values
K = options.strike_price.values
v = options.adj_bivs.values
t = options.tdays_to_expiry.values
d1 = np.log(F / K) + (r - q + 0.5 * v * v) * t
d1 /= np.sqrt(t) * v
options['vega'] = F * np.exp(-q * t) * norm.pdf(d1) * np.sqrt(t) * 0.01

def fit_qlr(dexp):
    
    x = np.log(dexp.strike_price / dexp.forward_price).values
    xx = x*x
    y = (dexp.adj_bivs ** 2).values
    
    lr = LinearRegression()
    lr = lr.fit(np.array([xx, x]).T, y, sample_weight=dexp.vega)
    
    return pd.Series(lr.coef_.tolist()[:1] + [dexp.atm_iv.values[0], x.min(), x.max()])

surfaces = options.groupby("dexpd").apply(fit_qlr)
surfaces = surfaces.reset_index()

cols = ['dexpd', 'date_current', 'expiration_date', 'tdays_to_expiry']
surfaces = surfaces.merge(options[cols].drop_duplicates(), on='dexpd', how='inner')
surfaces = surfaces.iloc[:, 1:]
surfaces.columns = ['a', 'c', 'xmin', 'xmax', 'date_current', 'expiration_date', 'tdays_to_expiry']

In [11]:
x = np.log(np.repeat(
    np.arange(70, 131, 1).reshape(1, -1) / 100,
    surfaces.shape[0],
    axis=0
))
xx = x * x

In [12]:
a = surfaces.a.values.reshape(-1, 1)
c = surfaces.c.values.reshape(-1, 1)
surface_data = pd.DataFrame(np.multiply(xx, a) + c)
surface_data['tdays_to_expiry'] = surfaces.tdays_to_expiry
surface_data['expiration_date'] = surfaces.expiration_date
surface_data['date_current'] = surfaces.date_current

In [44]:
%matplotlib tk
X = np.arange(70, 131, 1)

dates = surface_data.date_current.unique()

angles, pa = [], 15
for i, date in enumerate(dates):
    if pa == 65:
        d = -1
    elif pa == 15:
        d = 1
    pa += d
    angles.append(pa)
    
frames = list(zip(dates, angles))

fig = plt.figure(figsize=(20, 11))
ax = fig.add_subplot(111, projection='3d')

def plot_surface(n, X, ax, options):
    
    date, angle = n
    tmp = surface_data[surface_data.date_current == date]
    
    X = np.arange(70, 131, 1)
    Y = (tmp.tdays_to_expiry.values * 252).astype(int)
    X, Y = np.meshgrid(X, Y)
    Z = np.round(tmp.iloc[:, :61].values * 100, 2)
    
    ax.clear()
    surf = ax.plot_surface(X, Y, Z, cmap="viridis")
    
    ax.set_title(date, pad=10)
    ax.set_xlabel("Moneyness", labelpad=10)
    ax.set_ylabel("T Days to Expiry", labelpad=10)
    ax.set_zlabel("Implied Volatility", labelpad=10)
    ax.view_init(elev=30, azim=angle)
    
    return surf
    
surf_ani = animation.FuncAnimation(fig, plot_surface, fargs=(X, ax, options), frames=frames, interval=150, blit=False)
surf_ani.save("plots/surf_year_in_review.mp4")

In [45]:
surfaces.to_csv("data/surface_params.csv")

### 